In [1]:
import nltk
import json
import spacy
import re, urllib, json
from bs4 import BeautifulSoup
import time
from collections import Counter

In [4]:
from nltk import FreqDist
from nltk import ngrams


In [120]:
from google_ngram_downloader import readline_google_store
 
count = 0
fname, url, records = next(readline_google_store(ngram_len=1, indices=word[0]))
 
try:
	record = next(records)
 
	while record.ngram != word:
		record = next(records)
 
	while record.ngram == word:
		count = count + record.match_count
		record = next(records)
		
except StopIteration:
	pass


In [7]:
#read json file
with open('dev-v2.0.json') as json_data:
    content = json.load(json_data)
    json_data.close()

#parse json into a list of paragraphs
para = []
for data in content['data']:
    for paragraphs in data['paragraphs']:
        para.append(paragraphs['context'])

# #parse paragraph into list of sentences
# def parse_sentences(paragraph): 
#     sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
#     sentences = sent_detector.tokenize(paragraph.strip())
#     return sentences

# squad_sentences = []
# for p in para:
#     squad_sentences += parse_sentences(p)

FileNotFoundError: [Errno 2] No such file or directory: 'dev-v2.0.json'

In [3]:
# for s in squad_sentences:
#      if 'CARDINAL' in get_entity(s) or 'QUANTITY' in get_entity(s):
#             print(s)

In [8]:
nlp = spacy.load("en_core_web_lg")

In [21]:
def countable_noun(noun):
    url = 'https://books.google.com/ngrams/graph?content=many+' + noun + '%2C+much+' + noun + '&year_start=1800&year_end=2000'
    response = urllib.request.urlopen(url)
    html = response.read().decode('utf-8')

    try:
        many_data = json.loads(re.search('\{"ngram": "many ' + noun + '".*?\}', html).group(0))['timeseries']
        many = sum(many_data) / float(len(many_data))
    except:
        many = 0.0
    try:
        much_data = json.loads(re.search('\{"ngram": "much ' + noun + '".*?\}', html).group(0))['timeseries']
        much = sum(much_data) / float(len(much_data))
    except:
        much = 0.0
    if many > much:
        return True
    return False

In [44]:
def get_NE(sentence):
    l = []
    doc = nlp(sentence)
    for ent in doc.ents:
        l.append(ent.label_)
    return l
def get_nsubj(sentence):
    doc = nlp(sentence)
    for chunk in doc.noun_chunks:
        if chunk.root.dep_ == 'nsubj':
            return chunk.text
    return ''

def get_ROOT(sentence):
    doc = nlp(sentence)
    for token in doc:
        if token.dep_ == 'ROOT':
            return token.text, token.lemma_,token.tag_
    return ''

def get_entity(sentences):
    entity = []
    doc = nlp(sentences)
    for ent in doc.ents:
        entity.append( ent.label_)
    return entity


def get_namechunks(sentence):
    chunks = {}
    doc = nlp(sentence)
    for chunk in doc.noun_chunks:
        chunks[chunk.text] = chunk.root.text
        #chunks.append(chunk.text)
    return chunks

def get_tense(verb_t):
    # the verb is present third person singular
    if verb_t == 'VBZ':
        result = " does "
    elif verb_t in ['VBP', 'VBG', 'VB']:
        # tense = "present"
        result = " do " 
    elif verb_t in ['VBD', 'VBN']:
        # tense = "past"
        result = " did "
    else:
        result = ' does '
    return result

def get_pps(sentence):
    doc = nlp(sentence)
    pps = []
    for token in doc:
        # Try this with other parts of speech for different subtrees.
        if token.pos_ == 'ADP':
            pp = ' '.join([tok.orth_ for tok in token.subtree])
            pps.append(pp)
    return pps

In [23]:
def gen_question_type1(sentence,obj):
    root = get_ROOT(sentence)
    subj =get_nsubj(sentence)
    prep = ''
    loc = ''
    if len(get_pps(sentence)) > 0:
        for pp in get_pps(sentence):
            if pp.upper().split()[0] in ['DURING','IN','WHITIN','FROM','UNTIL'] and len(get_entity(pp))>0:
                if 'DATE' in get_entity(pp) or 'TIME' in get_entity(pp): #'CARDINAL' in get_entity(pp) or
                    prep = pp
            if (pp.upper().split()[0] == 'IN' or pp.upper().split()[0] == 'ON' or pp.upper().split()[0] == 'AT') and len(get_entity(pp))>0:
                if get_entity(pp)[0] in ['GPE','FAC','ORG','LOC']:
                    loc = pp
                    
    if subj != None:
        return "How many "+obj+get_tense(root[2])+subj+" "+root[1].lower()+' '+prep.lower()+' '+loc+'?'
    else:
        return "How many "+obj+" were "+root[0].lower()+' '+prep.lower()+' '+loc+'?'

In [24]:
def gen_question_type2(sentence,obj):
    prep = ''
    if len(get_pps(sentence)) > 0:
        for pp in get_pps(sentence):
            if pp.upper().split()[0] == 'IN' and len(get_entity(pp))>0:
                if get_entity(pp)[0] in ['GPE','FAC','ORG','LOC']:
                    prep = pp

    return "How many "+obj+" are there "+prep.lower()+'?'

In [25]:
def gen_question_type3(sentence,obj):
    root = get_ROOT(sentence)
    subj =get_nsubj(sentence)
    if obj == '' and root == '' :
        return ''
    if obj == '' and subj == '' :
        return ''
    if root == 'be':
        return ''
    prep = ''
    loc = ''
    if len(get_pps(sentence)) > 0:
        for pp in get_pps(sentence):
            if pp.upper().split()[0] in ['DURING','IN','WHITIN','FROM','UNTIL'] and len(get_entity(pp))>0:
                if 'CARDINAL' in get_entity(pp) or 'DATE' in get_entity(pp) or 'TIME' in get_entity(pp):
                    prep = pp
            if (pp.upper().split()[0] == 'IN' or pp.upper().split()[0] == 'ON') and len(get_entity(pp))>0:
                if get_entity(pp)[0] in ['GPE','FAC','ORG','LOC']:
                    loc = pp

    return "How much "+obj+get_tense(root[2])+subj+" "+root[1].lower()+' '+prep.lower()+' '+loc+'?'

In [26]:
def select_question(sentence):
    obj = ''
    for chunk in get_namechunks(sentence).keys():
        if 'CARDINAL' in get_entity(chunk) or 'QUANTITY' in get_entity(chunk):
            obj = get_namechunks(sentence)[chunk]

    if obj == '':
        return gen_question_type3(sentence, obj)
    
    if countable_noun(obj):
        if 'there' in sentence.lower():
            return gen_question_type2(sentence, obj)
        else:
            return gen_question_type1(sentence, obj)
    else:
        return gen_question_type3(sentence, obj)

In [53]:
def select_question(sentence):
    obj = ''
    ner = get_NE(sentence)
    if 'MONEY' in ner or 'CARDINAL' in ner or 'PERCENT' in ner or 'QUANTITY' in ner:
        for chunk in get_namechunks(sentence).keys():
            if 'CARDINAL' in get_entity(chunk) or 'QUANTITY' in get_entity(chunk):
                obj = get_namechunks(sentence)[chunk]

        if obj == '':
            return gen_question_type3(sentence, obj)
        
        if countable_noun(obj):
            if 'there' in sentence.lower():
                return gen_question_type2(sentence, obj)
            else:
                return gen_question_type1(sentence, obj)
        else:
            return gen_question_type3(sentence, obj)
    else:
        return ''

In [54]:
select_question(txt2)

'How much  do an anonymous alumnus donate  ?'

In [31]:
txt = 'The Duke has given up hope of a cure, but his wife Elizabeth persuades him to see Lionel Logue, an Australian speech therapist living in London'

In [41]:
txt2 = 'an anonymous alumnus donate $25,000 to the Tepper School Annual Fund.'

In [43]:
for chunk in get_namechunks(txt2).keys():
        if 'CARDINAL' in get_entity(chunk) or 'QUANTITY' in get_entity(chunk):
            obj = get_namechunks(sentence)[chunk]
        print(chunk)

an anonymous alumnus
the Tepper School Annual Fund


In [46]:
get_NE(txt2)

['MONEY', 'ORG']

In [27]:
select_question('The term itself was coined by eighteenth-century historians and the distinction between the Old Kingdom and the Early Dynastic Period is not one which would have been recognized by Ancient Egyptians.')

''

In [28]:
def open_html(filename):
    soup = BeautifulSoup(open(filename,encoding='utf-8'), "html.parser")
    page = soup.find_all('p')
    paragraphs = ''
    for p in page:
        paragraphs += p.getText()
    return paragraphs

def parse_sentences(paragraph): 
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    sentences = sent_detector.tokenize(paragraph.strip())
    return sentences

In [29]:
paragraphs = open_html("a2.htm")

In [56]:
for sentence in parse_sentences(paragraphs):
    if select_question(sentence) != '':
        print(sentence)
        print(select_question(sentence))
    #time.sleep(5)

The men become friends as they work together, and after his brother abdicates the throne, the new king relies on Logue to help him make his first wartime radio broadcast on Britain's declaration of war on Germany in 1939.Seidler read about George VI's life after overcoming a stuttering condition he endured during his youth.
How much  do The men become in 1939.seidler on Germany?
He later rewrote his screenplay for the stage to focus on the essential relationship between the two protagonists.
How many protagonists did He rewrite  ?
On a budget of £8 million, it earned over £250 million internationally ($400 million).
How much  did it earn  ?
The film received many awards and nominations, particularly for Colin Firth's performance; his Golden Globe Award for Best Actor was the sole win at that ceremony from seven nominations.
How many nominations did The film receive  ?
The King's Speech won seven British Academy Film Awards, including Best Picture, Best Actor (Firth), Best Supporting Ac

How much  did Variety estimate in march 2011 ?
Rotten Tomatoes gives the film a score of 95% based on reviews from 261 critics; their average rating was calculated as 8.6/10.
How many critics did Rotten Tomatoes calculate  ?
Metacritic gave the film a weighted score of 88/100, based on 41 critiques, which it ranks as "universal acclaim".
How many critiques did Metacritic give  ?
Empire gave the film five stars out of five, commenting, "You'll be lost for words."
How many stars did Empire give  ?
Roger Ebert of the Chicago Sun-Times awarded the film a full four stars, commenting that "what we have here is a superior historical drama and a powerful personal one."
How many stars did Roger Ebert award  ?
Peter Bradshaw of The Guardian gave four stars out of five, stating, "Tom Hooper's richly enjoyable and handsomely produced movie ... is a massively confident crowd-pleaser.
How many stars did Peter Bradshaw give  ?


HTTPError: HTTP Error 302: The HTTP server returned a redirect error that would lead to an infinite loop.
The last 30x error message was:
Found

In [126]:
get_ROOT('campaign')

('campaign', 'campaign', 'NN')

In [140]:
import pattern3

In [ ]:
import pluralize, singularize

In [142]:
pattern3.pluralize('child')

AttributeError: module 'pattern3' has no attribute 'pluralize'